# Prédiction du niveau de pollution dans le métro

# 1. Initialisation

In [23]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np
import re
import glob
import os
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [24]:
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [25]:
%flake8_on

# 2. Nettoyage des données

### 2.0 Création de la table jour - nom - type

In [26]:
cal = pd.read_csv('1 - Données brutes/calendrier.csv', sep=';')
cal['CAT_JOUR'] = 'xxx'
cal['date'] = cal.dates.str[-4:] + "-" + \
    cal.dates.str[3:5] + "-" + cal.dates.str[:2]
cal['date'] = pd.to_datetime(cal['date'])
cal.drop(columns={'dates'}, inplace=True)
cal['date'] = pd.arrays.DatetimeArray(cal['date']).date
for i in range(cal.shape[0]):
    if cal.jour.iloc[i] == 'dimanche' or cal.ferie.iloc[i] == 1:
        cal.CAT_JOUR.iloc[i] = 'DIJFP'
    else:
        if cal.jour.iloc[i] == 'samedi':
            if cal.vacances.iloc[i] == 'VS':
                cal.CAT_JOUR.iloc[i] = 'SAVS'
            else:
                cal.CAT_JOUR.iloc[i] = 'SAHV'
        else:
            if cal.vacances.iloc[i] == 'VS':
                cal.CAT_JOUR.iloc[i] = 'JOVS'
            else:
                cal.CAT_JOUR.iloc[i] = 'JOHV'

In [27]:
cal.head()

,jour,ferie,vacances,semestre,CAT_JOUR,date
0,mardi,1,HV,2013S1,DIJFP,2013-01-01
1,mercredi,0,HV,2013S1,JOHV,2013-01-02
2,jeudi,0,HV,2013S1,JOHV,2013-01-03
3,vendredi,0,HV,2013S1,JOHV,2013-01-04
4,samedi,0,HV,2013S1,SAHV,2013-01-05


### 2.1 Nettoyage des données de qualité de l'air en station

In [28]:
def load_and_clean_df_air(data):
    df_air = pd.read_csv(data, delimiter=';')

    # Conversion de la donnée temporelle en objet datetime
    df_air['DATE/HEURE'] = df_air['DATE/HEURE'].str.extract('([^+]*)')
    df_air['DATE/HEURE'] = pd.to_datetime(df_air['DATE/HEURE'])

    # Nettoyage des données numériques
    df_air = df_air.replace('ND', np.nan)
    df_air[['NO', 'PM10']] = df_air[['NO', 'PM10']].applymap(
        lambda x: re.search('(\d+)', x).group(1) if pd.notnull(x) else np.nan)

    df_air[['TEMP', 'HUMI']] = df_air[['TEMP', 'HUMI']].applymap(
        lambda x: x.replace(',', '.') if pd.notnull(x) else np.nan)

    df_air[df_air.columns[1:]] = df_air[df_air.columns[1:]].applymap(
        lambda x: x.replace('<', '')
        if pd.notnull(x) else np.nan)

    df_air[df_air.columns[1:]] = df_air[df_air.columns[1:]].astype(float)

    # Complétion de la série temporelle et passage en index
    df_air = df_air.groupby('DATE/HEURE').mean()
    full_index = pd.date_range(start=df_air.index[0],
                               end=df_air.index[-1], freq='H')
    df_air = df_air.reindex(full_index)

    # Interpolation pour "sauver" les lignes qui peuvent l'être
    df_air = df_air.sort_index()
    # On interpole jusqu'à 2 NaN consécutif. Au dela, on laisse les NaN
    df_air = df_air.interpolate(limit=2)
    # Rq : interpolate et limit fonctionnent tel que si on a un trou de
    # 10 valeurs il va interpoler le 2 premières en se basant sur
    # la précédente et la onzième

    return df_air

11:31: W605 invalid escape sequence '\d'


In [29]:
path = '1 - Données brutes/qualite_air_stations/'
files = ['qualite-de-lair-mesuree-dans-la-station-franklin-d-roosevelt.csv']
df_fro = load_and_clean_df_air(path + files[0])

In [30]:
df_fro['date'] = pd.arrays.DatetimeArray(df_fro.index).date
df_fro['heure'] = pd.arrays.DatetimeArray(df_fro.index).time

In [31]:
df_fro = df_fro.reset_index()
df_fro = df_fro.rename(columns={'index': 'date/heure'})

In [32]:
df_fro.head()

,date/heure,NO,NO2,PM10,CO2,TEMP,HUMI,date,heure
0,2013-01-01 02:00:00,23.0,41.0,26.0,612.0,15.1,65.6,2013-01-01,02:00:00
1,2013-01-01 03:00:00,43.0,61.0,18.0,536.0,14.9,65.0,2013-01-01,03:00:00
2,2013-01-01 04:00:00,22.0,52.0,45.0,490.0,14.9,63.3,2013-01-01,04:00:00
3,2013-01-01 05:00:00,27.0,56.0,22.0,481.0,14.9,63.7,2013-01-01,05:00:00
4,2013-01-01 06:00:00,27.0,57.0,15.0,464.0,15.0,63.3,2013-01-01,06:00:00


### 2.2 Nettoyage des données météo

In [33]:
def mysql_connect(query):
    try:
        connection = mysql.connector.connect(host='82.64.150.143',
                                             port='3306',
                                             database='sd701',
                                             user='teamsd701',
                                             password='TeamPlombierSD701')
        cursor = connection.cursor()
        cursor.execute(query)
        table_col = [i[0] for i in cursor.description]
        table_rows = cursor.fetchall()
        cursor.close()
        connection.close()

    except Error as e:
        print("Error while connecting to MySQL", e)

    return table_rows, table_col

In [34]:
query = ("SELECT * FROM weather_datas")
db = mysql_connect(query)
df_meteo = pd.DataFrame(db[0], columns=db[1])
df_meteo = df_meteo.replace('NaN', np.nan)
df_meteo['annee'] = df_meteo.unique_id.str[:4]
df_meteo['mois'] = df_meteo.unique_id.str[4:-5]
df_meteo['jour'] = df_meteo.unique_id.str[-5:-3]
mois = df_meteo.mois.unique().tolist()
ind = ['08', '04', '12', '02', '01', '07', '06', '05', '03', '11', '10', '09']
df_meteo.mois = df_meteo.mois.replace(mois, ind)
df_meteo.drop(columns={'date', 'unique_id'}, inplace=True)
df_meteo['date/heure'] = df_meteo['annee'] + "-" + df_meteo['mois'] + \
    "-" + df_meteo['jour'] + " " + df_meteo['heure'] + ":00:00"
df_meteo['date/heure'] = pd.to_datetime(df_meteo['date/heure'])
df_meteo = df_meteo.set_index('date/heure')
df_meteo.drop(columns={'heure', 'annee', 'mois', 'jour'}, inplace=True)
df_meteo = df_meteo.reset_index()
for i in range(1, len(df_meteo.columns)):
    df_meteo = df_meteo.rename(columns={df_meteo.columns[i]:
                                        df_meteo.columns[i] + '_ext'})
    coll = df_meteo.columns[i]
    df_meteo[coll] = df_meteo[coll].astype(np.float64)
df_meteo.head()

,date/heure,temp_ext,pluie_ext,vent_ext,rafale_ext,humidite_ext,ressenti_ext,radiation_ext,pt_rose_ext,pression_ext,visibilite_ext
0,2013-08-01 00:00:00,26.1,0.0,7.0,20.5,50.0,29.9,NaN,14.9,1010.1,20.0
1,2013-08-01 01:00:00,25.2,0.0,9.0,15.5,55.0,29.4,NaN,15.5,1010.2,20.0
2,2013-08-01 02:00:00,21.5,0.0,4.0,9.4,72.0,26.1,NaN,16.2,1015.9,20.0
3,2013-08-01 03:00:00,20.6,0.0,6.0,8.6,77.0,25.4,NaN,16.4,1015.6,20.0
4,2013-08-01 04:00:00,19.9,0.0,6.0,12.2,79.0,NaN,NaN,16.1,1015.1,20.0


### 2.3 Nettoyage des données de qualité de l'air extérieur

In [35]:
path = '1 - Données brutes/qualite_air_ext/'
dfs = []
for filename in glob.glob(path + '*.csv'):
    dfs.append(pd.read_csv(filename))
df_ext = pd.concat(dfs)
df_ext = df_ext[df_ext['nom_dept'] == 'PARIS']
df_ext = df_ext[(df_ext['id_com'] == 75102)
                | (df_ext['id_com'] == 75106)
                | (df_ext['id_com'] == 75104)]
df_ext = df_ext[['date_debut', 'id_com', 'nom_poll', 'valeur']]
df_ext.date_debut = df_ext.date_debut.str[:-3].replace('/', '-')
df_ext['date_debut'] = pd.to_datetime(df_ext['date_debut'])
df_ext.rename(columns={'date_debut': 'date/heure'}, inplace=True)
df_ext = df_ext.set_index('date/heure')
df_ext.drop(columns={'id_com'}, inplace=True)
df_ext = df_ext.groupby(by=['date/heure', 'nom_poll']).mean()
df_ext = df_ext.reset_index('nom_poll')
df_ext = df_ext.pivot_table(index=df_ext.index,
                            columns=['nom_poll'],
                            values='valeur',
                            aggfunc='sum')
df_ext = df_ext.rename(columns={'NO2': 'NO2_ext',
                                'NOX as NO2': 'NOX_as_NO2_ext',
                                'O3': 'O3_ext',
                                'PM10': 'PM10_ext',
                                'PM2.5': 'PM25_ext'})
df_ext = df_ext.reset_index()
df_ext.head()

nom_poll,date/heure,NO2_ext,NOX_as_NO2_ext,O3_ext,PM10_ext,PM25_ext
0,2017-09-18 16:00:00,65.000000,174.466667,40.0,NaN,NaN
1,2017-09-18 17:00:00,58.333333,157.100000,39.0,NaN,NaN
2,2017-09-18 18:00:00,68.000000,171.000000,35.0,NaN,NaN
3,2017-09-18 19:00:00,81.333333,170.333333,17.0,NaN,NaN
4,2017-09-18 20:00:00,78.333333,133.366667,9.0,NaN,NaN


### 2.5 Nettoyage des données de trafic ferroviaire

In [36]:
# Nombre de passages de trains théoriques en station sur l'heure précédente
# ex : le dimanche, 24 trains sont passés entre 00:00 et 01:00
df_trafic = pd.read_csv('1 - Données brutes/trafic.csv', sep=';')
df_trafic['heure'] = pd.to_datetime(df_trafic['heure'])
df_trafic['heure'] = pd.arrays.DatetimeArray(df_trafic.heure).time
df_trafic.head()

,heure,jour,trafic
0,00:00:00,dimanche,24
1,01:00:00,dimanche,24
2,02:00:00,dimanche,24
3,03:00:00,dimanche,0
4,04:00:00,dimanche,0


### 2.6 Nettoyage des données de validation aux bornes

- Pas de données disponibles sur S2 2019 et S1 2020
- Les données disponibles sont à la maille journée, ou bien par tranche horaire et jour type
- On utilise la maille tranche horaire/jour type/semestre prend la plage [S1 2016 - S1 2019] U [S2 2020 - S1 2021]
- Le pourcentage de validation est défini comme suit pour une station i : Rapport entre le nombre de validations à une station i, sur une tranche horaire TH, et le nombre de validations sur la journée entière à cette station i.
- Pour les catégories de jours :
> - JOHV : Jour Ouvré Hors Vacances Scolaires
> - SAHV : Samedi Hors Vacances Scolaires.
> - JOVS : Jour Ouvré en période de Vacances Scolaires.
> - SAVS : Samedi en période de Vacances Scolaires.
> - DIJFP : Dimanche et Jour Férié et les ponts
- L'heure indiquée dans le dataframe correspond à la plage qui vient de s'écouler

In [37]:
path = '1 - Données brutes/histo_validations/'
dfs = []
for filename in glob.glob(path + 'txt/*.txt'):
    df_temp = pd.read_csv(filename, sep='\t', low_memory=False)
    df_temp['semestre'] = filename[42:48]
    df_temp = df_temp[df_temp['LIBELLE_ARRET'] == 'FRANKLIN-D.ROOSEVELT']
    df_temp = df_temp[['semestre',
                       'CAT_JOUR',
                       'TRNC_HORR_60',
                       'pourc_validations']]
    dfs.append(df_temp)
df_val1 = pd.concat(dfs)

dfs2 = []
for filename in glob.glob(path + '*.csv'):
    df_temp2 = pd.read_csv(filename, sep=';', low_memory=False)
    df_temp2['semestre'] = filename[38:44]
    df_temp2 = df_temp2[df_temp2['LIBELLE_ARRET'] == 'FRANKLIN-D.ROOSEVELT']
    df_temp2 = df_temp2[['semestre',
                         'CAT_JOUR',
                         'TRNC_HORR_60',
                         'pourc_validations']]
    dfs2.append(df_temp2)
df_val2 = pd.concat(dfs2)
df_val = pd.concat([df_val1, df_val2])
df_val['heure'] = df_val.TRNC_HORR_60.str.split('-', expand=True)[1]
df_val['heure'] = df_val['heure'].replace('H', '', regex=True).astype(str)
df_val['heure'] = df_val['heure'] + ':00:00'
df_val = df_val[df_val.heure != 'None:00:00']
df_val['heure'] = pd.to_datetime(df_val['heure'])
df_val['heure'] = pd.arrays.DatetimeArray(df_val.heure).time
df_val = df_val[['semestre', 'CAT_JOUR', 'heure', 'pourc_validations']]
df_val = df_val.replace(',', '.', regex=True)
df_val = df_val.reset_index()
df_val.drop(columns={'index'}, inplace=True)
df_val['pourc_validations'] = df_val['pourc_validations'].astype(np.float64)
df_val.head()

,semestre,CAT_JOUR,heure,pourc_validations
0,2016S2,DIJFP,01:00:00,3.20
1,2016S2,DIJFP,11:00:00,1.46
2,2016S2,DIJFP,12:00:00,2.14
3,2016S2,DIJFP,13:00:00,3.76
4,2016S2,DIJFP,14:00:00,3.95


# 3. Création du dataframe global

### 3.1 Affichage des dataframes

In [43]:
cal.head(2)

,jour,ferie,vacances,semestre,CAT_JOUR,date
0,mardi,1,HV,2013S1,DIJFP,2013-01-01
1,mercredi,0,HV,2013S1,JOHV,2013-01-02


In [44]:
df_fro.head(2)

,date/heure,NO,NO2,PM10,CO2,TEMP,HUMI,date,heure
0,2013-01-01 02:00:00,23.0,41.0,26.0,612.0,15.1,65.6,2013-01-01,02:00:00
1,2013-01-01 03:00:00,43.0,61.0,18.0,536.0,14.9,65.0,2013-01-01,03:00:00


In [45]:
df_meteo.head(2)

,date/heure,temp_ext,pluie_ext,vent_ext,rafale_ext,humidite_ext,ressenti_ext,radiation_ext,pt_rose_ext,pression_ext,visibilite_ext
0,2013-08-01 00:00:00,26.1,0.0,7.0,20.5,50.0,29.9,NaN,14.9,1010.1,20.0
1,2013-08-01 01:00:00,25.2,0.0,9.0,15.5,55.0,29.4,NaN,15.5,1010.2,20.0


In [46]:
df_ext.head(2)

nom_poll,date/heure,NO2_ext,NOX_as_NO2_ext,O3_ext,PM10_ext,PM25_ext
0,2017-09-18 16:00:00,65.000000,174.466667,40.0,NaN,NaN
1,2017-09-18 17:00:00,58.333333,157.100000,39.0,NaN,NaN


In [47]:
df_trafic.head(2)

,heure,jour,trafic
0,00:00:00,dimanche,24
1,01:00:00,dimanche,24


In [14]:
df_val.head(2)

,semestre,CAT_JOUR,heure,pourc_validations
0,2016S2,DIJFP,01:00:00,3.20
1,2016S2,DIJFP,11:00:00,1.46


### 3.2 Concaténation

In [ ]:
df = pd.DataFrame()

In [ ]:
df = df_fro.merge(cal, on='date', how='left')
df = df.merge(df_meteo, on='date/heure', how='left')
df = df.merge(df_ext, on='date/heure', how='left')
df = df.merge(df_trafic, on=['jour', 'heure'], how='left')
df = df.merge(df_val, on=['semestre', 'CAT_JOUR', 'heure'], how='left')

column00 = df.pop('date/heure')
column0 = df.pop('date')
column1 = df.pop('heure')
column2 = df.pop('jour')
column3 = df.pop('ferie')
column4 = df.pop('vacances')
column5 = df.pop('semestre')
column6 = df.pop('CAT_JOUR')
df.insert(0, 'CAT_JOUR', column6)
df.insert(0, 'semestre', column5)
df.insert(0, 'vacances', column4)
df.insert(0, 'ferie', column3)
df.insert(0, 'jour', column2)
df.insert(0, 'heure', column1)
df.insert(0, 'date', column0)
df.insert(0, 'date/heure', column00)

df.head()

In [ ]:
df.info()

In [ ]:
# pip install pyarrow
df.to_parquet('2 - Données retraitées/dataset_metro.parquet',
              engine='pyarrow')

In [ ]:
# pd.read_parquet('2 - Données retraitées/dataset_metro.parquet')